<a href="https://colab.research.google.com/github/virtualthingz/LiveLessons/blob/master/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_%EA%B0%9C%EB%B0%9C%EA%B3%BC%EC%A0%95_12%EA%B0%95_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright &copy; 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Data Validation
***TensorFlow Extended의 주요 구성 요소의 예입니다***

참고: 지금 바로 주피터 스타일의 노트북에서 이 예제를 실행할 수 있습니다. 설정이 필요 없습니다! "Google Collab에서 실행"을 클릭하십시오.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/data_validation/tfdv_basic.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/data_validation/tfdv_basic.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
</table></div>

이 예제 콜랩 노트북은 TDV(TensorFlow Data Validation)를 사용하여 데이터 세트를 조사하고 시각화하는 방법을 보여 줍니다. 여기에는 기술 통계량 조사, 스키마 추론, 이상 징후 확인 및 수정, 데이터 세트의 드리프트 및 스큐 확인 등이 포함됩니다. 운영 파이프라인에서 시간이 지남에 따라 변경되는 방식을 포함하여 데이터셋의 특성을 이해하는 것이 중요합니다. 또한 데이터의 이상 징후를 찾아 교육, 평가 및 서비스 데이터 세트를 비교하여 데이터 세트가 일관성이 있는지 확인하는 것도 중요합니다.

시카고 시에서 공개한 [택시 여행 데이터 세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)의 데이터를 사용합니다.

참고: 이 사이트는 시카고 시의 공식 웹사이트인 원래 소스인 www.cityofchicago.org에서 사용하도록 수정된 데이터를 사용하여 응용 프로그램을 제공합니다. 시카고 시는 본 사이트에 제공된 데이터의 내용, 정확성, 적시성 또는 완전성에 대해 어떠한 주장도 하지 않습니다. 이 사이트에 제공된 데이터는 언제든지 변경될 수 있습니다. 이 사이트에 제공된 데이터는 자신의 위험을 감수하고 사용 중인 것으로 파악됩니다.

[더 읽어보기](https://cloud.google.com/bigquery/public-data/chicago-taxi)에서는 [Google BigQuery](https://cloud.google.com/bigquery/))의 데이터 세트에 대해 설명합니다. [BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips)에서 전체 데이터 세트를 살펴보십시오.

요점: 모델 제작자 및 개발자로서, 이 데이터가 어떻게 사용되고 모델의 예측이 초래할 수 있는 잠재적인 이점과 해를 생각해 보십시오. 이와 같은 모델은 사회적 편견과 차별성을 강화할 수 있습니다. 어떤 기능이 당신이 해결하고자 하는 문제와 관련이 있습니까, 아니면 편견을 도입합니까? 자세한 내용은 [ML 공정성](https://developers.google.com/machine-learning/fairness-overview/)을 참조하십시오.

The columns in the dataset are:
<table>
<tr><td>pickup_community_area</td><td>fare</td><td>trip_start_month</td></tr>

<tr><td>trip_start_hour</td><td>trip_start_day</td><td>trip_start_timestamp</td></tr>
<tr><td>pickup_latitude</td><td>pickup_longitude</td><td>dropoff_latitude</td></tr>
<tr><td>dropoff_longitude</td><td>trip_miles</td><td>pickup_census_tract</td></tr>
<tr><td>dropoff_census_tract</td><td>payment_type</td><td>company</td></tr>
<tr><td>trip_seconds</td><td>dropoff_community_area</td><td>tips</td></tr>
</table>

### Pip을 업그레이드하십시오.

로컬로 실행할 때 시스템에서 Pip을 업그레이드하지 않도록 하려면 Collab에서 실행 중인지 확인하십시오. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 텐서 플로우를 설치합니다.

**참고: Google Collab에서는 패키지 업데이트로 인해 이 셀을 처음 실행할 때 런타임(Runtime > Restart runtime...)을 다시 시작해야 합니다.**

In [ ]:
!pip install tensorflow==2.2.0

## Check the Python version

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

## TFDV를 설치합니다.

이렇게 하면 모든 의존성을 끌어당길 수 있습니다. 1분 정도 걸립니다. 호환되지 않는 종속성 버전에 대한 경고 또는 오류는 무시하십시오.

**참고: Google Collab에서는 패키지 업데이트로 인해 이 셀을 처음 실행할 때 런타임(Runtime > Restart runtime...)을 다시 시작해야 합니다.**

In [ ]:
import tensorflow as tf

print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

## 런타임을 다시 시작했습니까?

위의 셀을 처음 실행할 때 Google Collab을 사용하는 경우, 런타임(Runtime > Restart runtime...)을 재시작해야 합니다. 이것은 Colab이 패키지를 로드하는 방식 때문입니다.

## 파일을 로드합니다.
Seagate는 Google 클라우드 스토리지에서 데이터 세트를 다운로드할 것입니다.

In [ ]:
import os
import tempfile, urllib, zipfile

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R {os.path.join(BASE_DIR, 'data')}

### 버전을 체크합니다

In [ ]:
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

## 통계를 계산하고 시각화합니다.

먼저 교육 데이터에 대한 통계를 계산하기 위해 ['tfdv.generate_statistics_from_csv'](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv)를 사용합니다. (스냅 경고를 무시합니다.)

TFDV는 존재하는 특징과 그 가치 분포의 형태에 대한 데이터를 간략히 설명하는 [통계](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto)를 계산할 수 있습니다.

내부적으로 TFDV는 [Apache Beam](https://beam.apache.org/)'s 데이터 전송 처리 프레임워크를 사용하여 대규모 데이터 세트에 대한 통계 계산을 확장합니다. TFDV와 더 깊이 통합하려는 애플리케이션의 경우(예: 데이터 생성 파이프라인 끝에 통계 생성을 첨부) API는 통계 생성을 위한 Beam PT 변환도 제공합니다.

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

이제 [Facets](https://pair-code.github.io/facets/)를 사용하여 교육 데이터의 간결한 시각화를 만들어 보겠습니다.

* 숫자 형상과 범주는 별도로 시각화되며 각 형상에 대한 분포를 보여주는 차트가 표시됩니다.
* 결측값 또는 영(0) 값을 가진 형상은 해당 형상의 예에 문제가 있을 수 있다는 시각적 지표로 백분율을 빨간색으로 표시합니다. 백분율은 해당 기능에 대한 결측값이 있거나 값이 0인 예제의 백분율입니다.
* pickup_census_tract 값이 있는 예는 없습니다. 이것은 치수 축소를 위한 기회입니다!
* 차트 위에 있는 "확장"을 클릭하여 디스플레이를 변경해 보십시오.
* 차트의 막대 위를 맴돌면 버킷 범위와 카운트가 표시됩니다.
* 로그 스케일과 선형 스케일 간을 전환해 보십시오. 로그 스케일이 'payment_type' 범주형 기능에 대해 훨씬 더 자세히 보여지는 것을 볼 수 있습니다.
* "Chart to show"(차트 표시) 메뉴에서 "quantiles"(정량화)를 선택하고 마커 위로 마우스를 가져가면 정량화 백분율이 표시됩니다.

In [ ]:
tfdv.visualize_statistics(train_stats)

## 스키마를 추론합니다.

이제 ['tfdv.dvp_twin'](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema)을 사용하여 데이터에 대한 스키마를 만들어 보겠습니다. 스키마는 ML과 관련된 데이터에 대한 제약 조건을 정의합니다. 예제 제약 조건에는 각 기능의 데이터 유형(숫자인지 범주형인지 또는 데이터에 존재하는 빈도)이 포함됩니다. 범주형 기능의 경우 스키마는 도메인 - 허용 가능한 값 목록도 정의합니다. 스키마를 쓰는 것은 특히 기능이 많은 데이터 세트의 경우 지루한 작업이 될 수 있으므로, TFDV는 기술 통계량을 기반으로 스키마의 초기 버전을 생성하는 방법을 제공합니다.

나머지 생산 파이프라인은 TFDV가 올바르게 생성하는 스키마에 의존하기 때문에 스키마를 올바르게 만드는 것이 중요합니다. 스키마는 데이터에 대한 문서도 제공하므로 여러 개발자가 동일한 데이터에 대해 작업할 때 유용합니다. 검토할 수 있도록 ['tfdv.display_display'](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema)를 사용하여 추론 스키마를 표시해 보겠습니다.

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## 평가 데이터의 오류를 확인합니다.

지금까지는 교육 데이터만 조사했습니다. 평가 데이터가 동일한 스키마를 사용하는 것을 포함하여 교육 데이터와 일관성이 있어야 합니다. 평가 데이터가 훈련 데이터와 거의 동일한 범위의 값 예제를 포함하는 것도 중요합니다. 따라서 평가 중 손실 표면에 대한 적용 범위는 훈련 중과 거의 동일합니다. 범주형 기능에 대해서도 마찬가지입니다. 그렇지 않으면, 손실 표면의 일부를 평가하지 않았기 때문에 평가 중에 식별되지 않는 교육 문제가 발생할 수 있습니다.

* 이제 각 기능에는 교육 데이터 세트와 평가 데이터 세트에 대한 통계가 포함됩니다.
* 이제 차트에는 교육 데이터 세트와 평가 데이터 세트가 모두 중첩되어 있으므로 쉽게 비교할 수 있습니다.
* 이제 차트에는 로그 또는 기본 선형 척도와 결합할 수 있는 백분율 보기가 포함됩니다.
* 'trip_miles'의 평균과 중위수는 교육 데이터 세트와 평가 데이터 세트에 따라 다릅니다. 그것이 문제를 일으킬까요?
* 와, 최대 '팁'은 교육 데이터 세트와 평가 데이터 세트에 비해 매우 다릅니다. 그것이 문제를 일으킬까요?
* 숫자 특성 차트에서 확장을 클릭하고 로그 척도를 선택합니다. 'trip_seconds' 기능을 검토하고 최대치의 차이를 확인합니다. 평가 결과 손실 표면의 일부가 누락됩니까?

In [ ]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

## 평가 이상 여부를 확인합니다.

평가 데이터 세트가 교육 데이터 세트의 스키마와 일치합니까? 이는 허용 가능한 값의 범위를 식별하려는 범주형 기능에 특히 중요합니다.

요점: 교육 데이터 세트에 없는 범주형 기능 값을 사용하여 데이터를 평가하려고 하면 어떻게 됩니까? 교육 데이터 세트의 범위를 벗어난 수치적 기능은 어떻습니까?

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## 스키마의 평가 이상을 수정합니다.

앗! 교육 자료에는 없던 평가 자료에는 회사라는 새로운 가치관이 들어 있는 것 같습니다. 우리는 'payment_type'에 대한 새로운 가치도 가지고 있습니다. 이러한 현상은 이상 징후로 간주되어야 하지만, 이러한 이상 징후에 대해 우리가 하기로 결정한 것은 데이터에 대한 도메인 지식에 따라 다릅니다. 문제가 실제로 데이터 오류를 나타내는 경우 기본 데이터를 수정해야 합니다. 그렇지 않으면 스키마를 업데이트하여 평가 데이터 세트에 값을 포함시킬 수 있습니다.

요점: 만약 우리가 이 문제들을 고치지 않는다면 우리의 평가 결과는 어떤 영향을 받을까요?

평가 데이터 세트를 변경하지 않는 한 모든 것을 수정할 수는 없지만, 스키마에서 쉽게 받아들일 수 있는 것은 수정할 수 있습니다. 여기에는 특정 기능에 대한 이상 징후와 이상 징후를 파악할 수 있는 여유와 범주적 기능에 대한 결측값을 포함하도록 스키마를 업데이트하는 작업이 포함됩니다. TFDV를 통해 해결해야 할 사항을 파악할 수 있었습니다.

지금 수정하고 다시 한 번 검토하겠습니다.

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

이제 교육 및 평가 데이터가 일관성이 있음을 확인했습니다! 감사합니다 TFDV;)

## 스키마 환경

또한 이 예에서는 'serving' 데이터 세트를 분리하므로 이것도 확인해야 합니다. 기본적으로 파이프라인의 모든 데이터 집합은 동일한 스키마를 사용해야 하지만 예외가 종종 있습니다. 예를 들어, 지도 학습에서 우리는 데이터 세트에 레이블을 포함해야 하지만 추론을 위해 모델을 지원할 때는 레이블이 포함되지 않습니다. 경우에 따라서는 약간의 스키마 변형이 필요합니다.

**환경*은 이러한 요구사항을 표현하는 데 사용할 수 있습니다. 특히 스키마의 기능은 'default_environment', 'in_environment', 'not_in_environment'를 사용하는 환경 집합과 연결될 수 있습니다.

예를 들어, 이 데이터 집합에서는 'tips' 피쳐가 학습용 레이블로 포함되지만 서비스 데이터에는 없습니다. 환경을 지정하지 않으면 예외로 표시됩니다.

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

아래 'tips' 피쳐을 다루겠습니다. 우리는 또한 우리의 스키마가 플로트를 기대했던 여행 초의 INT 값을 가지고 있습니다. TFDV는 이러한 차이를 인식하도록 함으로써 학습 및 서비스를 위해 데이터가 생성되는 방식의 불일치를 발견하는 데 도움이 됩니다. 모델 성능이 저하될 때까지 이러한 문제를 인지하기란 매우 쉽습니다. 때로는 치명적인 결과를 초래할 수도 있습니다. 중요한 문제가 될 수도 있고 아닐 수도 있지만, 어떤 경우에도 추가 조사를 위해 이러한 문제가 발생해야 합니다.

이 경우 INT 값을 플로트로 안전하게 변환할 수 있으므로 TFDV에 스키마를 사용하여 유형을 추론하도록 지시하고자 합니다. 지금 그렇게 해요.

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

이제 'tips' 기능(우리의 레이블)이 예외('Column droped')로 표시됩니다. 물론 서비스 데이터에 레이블이 지정될 것으로 예상되지 않으므로 TFDV에 이를 무시하라고 전달하겠습니다.

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## 드리프트와 스큐를 체크합니다

TFDV는 데이터 세트가 스키마에 설정된 기대치를 준수하는지 확인하는 것 외에도 드리프트 및 스큐를 감지하는 기능도 제공합니다. TFDV는 스키마에 지정된 드리프트/스큐 비교기를 기반으로 여러 데이터 세트의 통계를 비교하여 이 검사를 수행합니다.

### 드리프트

드리프트 감지는 범주형 기능과 연속된 데이터 범위(즉, 학습 데이터의 다른 요일 간) 간에 지원됩니다. 드리프트를 [L-infinity distance](https://en.wikipedia.org/wiki/Chebyshev_distance),)로 표현하며, 드리프트가 허용 가능한 것보다 높을 때 경고를 받도록 임계값 거리를 설정할 수 있습니다. 올바른 거리를 설정하는 것은 일반적으로 도메인 지식과 실험이 필요한 반복적인 프로세스입니다.

### 스큐

TFDV는 데이터에서 스키마 스큐, 피쳐 스큐 및 배포 스큐의 세 가지 스큐를 탐지할 수 있습니다.

#### 스키마 스큐

스키마 스큐는 학습 및 서비스 데이터가 동일한 스키마를 준수하지 않을 때 발생합니다. 학습 데이터와 서비스 데이터는 모두 동일한 스키마를 준수할 것으로 예상됩니다. 둘 사이의 예상되는 편차(예: 레이블 기능은 교육 데이터에만 있을 뿐 서비스에는 없을 것)는 스키마의 환경 필드를 통해 지정해야 합니다.

### 피쳐 스큐

피쳐 스큐는 모델이 훈련하는 피쳐 값이 서비스 제공 시간에 표시되는 피쳐 값과 다를 때 발생합니다. 예를 들어 다음과 같은 경우에 발생할 수 있습니다.

* 일부 기능 값을 제공하는 데이터 소스는 교육 및 서비스 시간 간에 수정됩니다.
* 훈련과 서비스 간에는 다양한 기능을 생성하는 논리가 있습니다. 예를 들어 일부 변환을 두 코드 경로 중 하나에만 적용하는 경우입니다.

### 분포 스큐

분포 왜곡은 학습 데이터 세트의 분포가 서비스 데이터 세트의 분포와 유의하게 다를 때 발생합니다. 분포 스큐의 주요 원인 중 하나는 서로 다른 코드 또는 다른 데이터 소스를 사용하여 교육 데이터 세트를 생성하는 것입니다. 또 다른 이유는 교육할 서비스 데이터의 비대표 하위 표본을 선택하는 잘못된 샘플링 메커니즘입니다.

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

이 예에서는 드리프트를 볼 수 있지만, 이는 우리가 설정한 임계값에 훨씬 못 미치는 수준입니다.

## 스키마를 고정합니다.

스키마가 검토되고 큐레이션되었으므로 "동결" 상태를 반영하여 파일에 저장할 것입니다.

In [ ]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

## TFDV 사용 시기입니다.

여기서처럼 TFDV는 교육 파이프라인의 시작에만 적용된다고 생각하기 쉬우나, 실제로 많은 용도로 사용됩니다. 몇 가지 추가 사항이 있습니다.

* 새로운 데이터를 검증하여 갑자기 불량 기능을 수신하기 시작하지 않았는지 확인합니다.
* 모델이 의사 결정 표면의 해당 부분에 대해 교육을 받았는지 확인하기 위해 추론을 위해 새 데이터를 검증합니다.
* 데이터를 변환하고 피쳐 엔지니어링을 수행한 후 데이터를 검증합니다([TensorFlow Transform](https://www.tensorflow.org/tfx/transform/))을 사용하여 잘못한 것이 없는지 확인합니다.